# Notebook 3: Cleaning and consolidating data related to cartels

In [1]:
import pandas as pd
import numpy as np
import glob
import datetime

In [2]:
icews_cartels = pd.read_csv('../data/icews_cartels.csv', parse_dates=['event_date'],dtype={
       'event_id':'int','intensity':'float','story_id':'int','sentence_number':'int'})

In [3]:
icews_cartels["event_text"].value_counts()

Use unconventional violence                            997
Use conventional military force                        354
fight with small arms and light weapons                101
Abduct, hijack, or take hostage                         67
Assassinate                                             32
Occupy territory                                        16
Kill by physical assault                                14
Torture                                                  8
Engage in mass killings                                  7
Conduct suicide, car, or other non-military bombing      2
Employ aerial weapons                                    1
fight with artillery and tanks                           1
Name: event_text, dtype: int64

In [18]:
icews_cartels["event_year"].value_counts()

2010    297
2011    219
2009    174
2008    172
2014    143
2007    137
2012    135
2013     65
2005     53
2019     50
2017     42
2006     42
2018     27
2004     17
1999      7
2020      4
1997      4
2002      4
2000      3
2003      2
1998      2
1996      1
Name: event_year, dtype: int64

In [4]:
#viewing different events to see major sources of event activities
mex_occupy = icews_cartels.loc[icews_cartels["event_text"].isin(['Occupy territory'])]
mex_occupy.head(10)

,Unnamed: 0,Unnamed: 0.1,event_id,event_date,source_name,source_sectors,source_country,event_text,cameo_code,intensity,...,story_id,sentence_number,publisher,city,district,state,country,lat,lon,event_year
227,13422718,13422718,20774982,2014-01-16,Knights Templar Cartel,"Dissident,Criminals / Gangs",Mexico,Occupy territory,192,-9.5,...,32588437,5,Associated Press Newswires,Tancitaro,NaN,Estado de Michoacan de Ocampo,Mexico,19.3377,-102.3620,2014
267,16337700,16337700,35016053,2019-10-22,Sinaloa Cartel,"Dissident,Criminals / Gangs",Mexico,Occupy territory,192,-9.5,...,50539757,4,Reuters News,Culiacan,NaN,Estado de Sinaloa,Mexico,24.8074,-107.3970,2019
269,16337752,16337752,35017144,2019-10-22,Sinaloa Cartel,"Dissident,Criminals / Gangs",Mexico,Occupy territory,192,-9.5,...,50539869,2,Reuters News,Culiacan,NaN,Estado de Sinaloa,Mexico,24.8074,-107.3970,2019
438,12402962,12402962,18977505,2012-08-23,Miguel Treviño Morales,"Criminals / Gangs,Dissident",Mexico,Occupy territory,192,-9.5,...,17200980,3,Associated Press Newswires,Mexico City,NaN,Distrito Federal,Mexico,19.4285,-99.1277,2012
495,4302126,4302126,6280294,2003-07-15,Drug Gang (Mexico),"Criminals / Gangs,Dissident",Mexico,Occupy territory,192,-9.5,...,13668451,2,Agence France-Presse,Nuevo Laredo,NaN,Estado de Tamaulipas,Mexico,27.4763,-99.5164,2003
528,7355038,7355038,10579504,2006-08-20,Drug Gang (Mexico),"Dissident,Criminals / Gangs",Mexico,Occupy territory,192,-9.5,...,14650943,2,El Universal,Sinaloa,NaN,Estado de Chiapas,Mexico,15.8931,-92.1298,2006
529,7360972,7360972,10585402,2006-08-22,Drug Gang (Mexico),"Dissident,Criminals / Gangs",Mexico,Occupy territory,192,-9.5,...,14653773,2,El Universal,Sinaloa,NaN,Estado de Chiapas,Mexico,15.8931,-92.1298,2006
674,9835927,9835927,14221307,2009-01-29,Drug Gang (Mexico),"Dissident,Criminals / Gangs",Mexico,Occupy territory,192,-9.5,...,15098980,3,Mural,Guadalajara,NaN,Estado de Jalisco,Mexico,20.6668,-103.3920,2009
675,9838931,9838931,14226178,2009-01-30,Drug Gang (Mexico),"Dissident,Criminals / Gangs",Mexico,Occupy territory,192,-9.5,...,15099439,4,Reforma,NaN,NaN,NaN,Mexico,19.4285,-99.1277,2009
711,10250534,10250534,14759579,2009-06-29,Drug Gang (Mexico),"Dissident,Criminals / Gangs",Mexico,Occupy territory,192,-9.5,...,15118826,2,El Norte,Guanajuato,NaN,Estado de Chiapas,Mexico,15.1059,-92.2526,2009


In [5]:
#explore source sectors for consolidation
icews_cartels["source_sectors"].unique()

array([nan, 'Dissident,Criminals / Gangs', 'Criminals / Gangs,Dissident',
       'General Population / Civilian / Social,Social',
       'Social,General Population / Civilian / Social',
       'Protestors / Popular Opposition / Mobs,Dissident',
       'Government,Military', 'Police,Government', 'Government,Police',
       'Government,Army,Military', 'Government,Marines,Military',
       'Military,Government', 'Unidentified Forces', 'Social,Business',
       'Military Intelligence,Military,Government',
       'Navy,Military,Government', 'Military,Marines,Government',
       'Marines,Government,Military', 'Military,Government,Marines',
       'Marines,Military,Government',
       'Justice / Law Ministry,Executive,Government,Drugs Ministry',
       'Local,Provincial,Government', 'Government',
       'Government,Police,National / Border Divisions',
       'Government,Executive,Foreign Ministry',
       'Government,Military,Army',
       'Military,Government,Military Intelligence',
       '

COMBINE AND CLEAN SOURCE SECTORS

In [6]:
source_sectors = {
    "Government,Police": "Government/Military/Police",
    "Police,Government": "Government/Military/Police",
    "Government,Police,National / Border Divisions": "Government/Military/Police",
    "Government,Military": "Government/Military/Police",
    "Military,Government": "Government/Military/Police",
    "Government,Marines,Military": "Government/Military/Police",
    "Military Intelligence,Military,Government": "Government/Military/Police",
    "Navy,Military,Government": "Government/Military/Police",
    "Army,Government,Military": "Government/Military/Police",
    "Military,Government,Army": "Government/Military/Police",
    "Government,Army,Military": "Government/Military/Police",
    "Government,Military,Army": "Government/Military/Police",
    "Marines,Government,Military": "Government/Military/Police",
    "Military,Government,Military Intelligence": "Government/Military/Police",
    "Army,Military,Government": "Government/Military/Police",
    "Military,Navy,Government": "Government/Military/Police",
    "Government,Military,Navy": "Government/Military/Police",
    "Marines,Military,Government": "Government/Military/Police",
    "Military,Government,Navy": "Government/Military/Police",
    "Government,Navy,Military": "Government/Military/Police",
    "Military,Marines,Government": "Government/Military/Police",
    "Government,Military,Marines": "Government/Military/Police",
    "Military,Government,Marines": "Government/Military/Police",
    "Justice / Law Ministry,Executive,Government,Drugs Ministry": "Government/Military/Police",
    "Local,Provincial,Government": "Government/Military/Police",
    "Government,Executive,Foreign Ministry": "Government/Military/Police",
    "Social,General Population / Civilian / Social": "Civilian",
    "General Population / Civilian / Social,Social": "Civilian",
    "Dissident,Criminals / Gangs": "Criminals/Dissidents",
    "Social,Business": "Civilian",
    "Protestors / Popular Opposition / Mobs,Dissident": "Civilian",
    "Criminals / Gangs,Dissident": "Criminals/Dissidents",
    "Government":"Government/Military/Police"}
icews_cartels = icews_cartels.replace(source_sectors)

In [7]:
#explore source sectors for consolidation
icews_cartels["source_sectors"].unique()

array([nan, 'Criminals/Dissidents', 'Civilian',
       'Government/Military/Police', 'Unidentified Forces'], dtype=object)

In [8]:
icews_cartels["source_sectors"].value_counts()

Criminals/Dissidents          1084
Government/Military/Police     325
Civilian                        96
Unidentified Forces             14
Name: source_sectors, dtype: int64

In [9]:
#viewing null source sectors
nan_sector = icews_cartels.loc[icews_cartels["source_sectors"].isna()]
nan_sector["source_name"].value_counts()

Mexico                          74
United States                    3
La Familia Michoacana Cartel     2
Central America                  1
Beltrán-Leyva Cartel             1
Name: source_name, dtype: int64

In [10]:
#check what to designate US military force
usa = nan_sector.loc[nan_sector["source_name"].isin(['United States'])]
usa.head(10)

,Unnamed: 0,Unnamed: 0.1,event_id,event_date,source_name,source_sectors,source_country,event_text,cameo_code,intensity,...,story_id,sentence_number,publisher,city,district,state,country,lat,lon,event_year
1172,7351221,7351221,10573342,2006-08-18,United States,NaN,United States,Use conventional military force,190,-10.0,...,14647892,1,Inter Press Service,Tijuana,NaN,Estado de Baja California,Mexico,32.5027,-117.0040,2006
1359,10607530,10607530,15409140,2009-12-26,United States,NaN,United States,Use conventional military force,190,-10.0,...,9852804,5,The Washington Post,NaN,NaN,NaN,Mexico,19.4285,-99.1277,2009
1360,10608862,10608862,15409670,2009-12-27,United States,NaN,United States,Use conventional military force,190,-10.0,...,13118918,5,The Washington Post,NaN,NaN,NaN,Mexico,19.4285,-99.1277,2009


In [11]:
nan_sector["event_text"].value_counts()

Use conventional military force    78
Use unconventional violence         3
Name: event_text, dtype: int64

In [12]:
#fill in null values
icews_cartels['source_sectors'] = icews_cartels['source_sectors'].fillna(icews_cartels['source_name'].map({'Mexico':'Government/Military/Police', 'La Familia Michoacana Cartel':'Criminals/Dissidents', 'United States':'Government/Military/Police', 'Central America':'Government/Military/Police'}))

In [13]:
#checking that it worked
nan_check = icews_cartels.loc[icews_cartels["source_name"].isin(['United States'])]
nan_check.head()

,Unnamed: 0,Unnamed: 0.1,event_id,event_date,source_name,source_sectors,source_country,event_text,cameo_code,intensity,...,story_id,sentence_number,publisher,city,district,state,country,lat,lon,event_year
1172,7351221,7351221,10573342,2006-08-18,United States,Government/Military/Police,United States,Use conventional military force,190,-10.0,...,14647892,1,Inter Press Service,Tijuana,NaN,Estado de Baja California,Mexico,32.5027,-117.0040,2006
1359,10607530,10607530,15409140,2009-12-26,United States,Government/Military/Police,United States,Use conventional military force,190,-10.0,...,9852804,5,The Washington Post,NaN,NaN,NaN,Mexico,19.4285,-99.1277,2009
1360,10608862,10608862,15409670,2009-12-27,United States,Government/Military/Police,United States,Use conventional military force,190,-10.0,...,13118918,5,The Washington Post,NaN,NaN,NaN,Mexico,19.4285,-99.1277,2009


In [14]:
#explore source sectors for consolidation
icews_cartels["source_name"].unique()

array(['Beltrán-Leyva Cartel', 'Juárez Cartel', 'Tijuana Cartel',
       'Sinaloa Cartel', 'Los Zetas Cartel', 'Gulf Cartel',
       'La Familia Michoacana Cartel', 'Knights Templar Cartel',
       'Attacker (Mexico)', 'Citizen (Mexico)', 'Criminal (Mexico)',
       'Citizen (United States)', 'Drug Gang (Mexico)',
       'Joaquín Guzmán Loera', 'Mexico', 'Mob (Mexico)',
       'Military (Mexico)', 'Police (Mexico)', 'Barrio Azteca Drug Gang',
       'Military Personnel (United States)',
       'Military Personnel - Special (Mexico)', 'Mexican Army',
       'Edgar Valdez Villarreal', 'Marine Corp (Mexico)',
       'Armed Gang (Mexico)', 'Business (Mexico)',
       'Miguel Treviño Morales', 'Secret Agent (Mexico)', 'Mexican Navy',
       'Militia (Mexico)', 'Military Personnel (Mexico)', 'Men (Mexico)',
       'Drug Dealer (Mexico)', 'Drug Gang (Brazil)', 'Mexicles Drug Gang',
       'Drug Enforcement Administration', 'Drug Dealer (Brazil)',
       'Governor (Mexico)', 'Police (Brazil)',

In [15]:
# what states are represented
icews_cartels["state"].unique()

array(['Estado de Zacatecas', 'Estado de Chihuahua', nan,
       'Estado de Yucatan', 'Estado de Chiapas',
       'Estado de Baja California', 'Estado de Tamaulipas',
       'Estado de Quintana Roo', 'Estado de Michoacan de Ocampo',
       'Distrito Federal', 'Estado de Hidalgo',
       'Estado de San Luis Potosi', 'Estado de Nuevo Leon',
       'Central America', 'Estado de Mexico', 'Estado de Sinaloa',
       'Estado de Tabasco', 'Estado de Veracruz-Llave',
       'Estado de Guerrero', 'Estado de Puebla', 'Estado de Morelos',
       'Estado de Tlaxcala', 'Estado de Oaxaca', 'Estado de Jalisco',
       'Estado de Sonora', 'Estado de Campeche', 'Estado de Guanajuato',
       'Estado de Coahuila de Zaragoza', 'Estado de Baja California Sur',
       'Estado de Querétaro'], dtype=object)

In [16]:
distrito_federal = icews_cartels.loc[icews_cartels["state"].isin(['Distrito Federal'])]
distrito_federal["city"].value_counts()

Mexico City     290
Villahermosa      3
Name: city, dtype: int64

In [17]:
icews_cartels["event_year"].value_counts()

2010    297
2011    219
2009    174
2008    172
2014    143
2007    137
2012    135
2013     65
2005     53
2019     50
2017     42
2006     42
2018     27
2004     17
1999      7
2020      4
1997      4
2002      4
2000      3
2003      2
1998      2
1996      1
Name: event_year, dtype: int64

In [14]:
icews_cartels["target_name"].unique()

array(['Foreign Affairs (Japan)', 'Citizen (Mexico)', 'Men (Mexico)',
       'Gulf Cartel', 'Mexico', 'Drug Gang (Mexico)',
       'Joaquín Guzmán Loera', 'Mob (Mexico)', 'Criminal (Mexico)',
       'Police (Mexico)', 'Military (Mexico)', 'Sinaloa Cartel',
       'Los Zetas Cartel', 'Mexican Navy',
       'Other Authorities / Officials (Mexico)', 'Juárez Cartel',
       'Children (Mexico)', 'Federal Police',
       'La Familia Michoacana Cartel', 'Barrio Azteca Drug Gang',
       'Farm Worker (Mexico)', 'Women (Mexico)',
       'Illegal Immigrant (Mexico)', 'Immigrants (Mexico)',
       'Marine Corp (Mexico)', 'Jaime Zapata', 'Detainee (Mexico)',
       'Criminal (Gulf Cartel)', 'Violent Group (Mexico)',
       'Business (Mexico)', 'Immigrants (Brazil)',
       'Secret Agent (Mexico)', 'Governor (Mexico)', 'Central America',
       'Military Personnel (Mexico)',
       'Military Personnel - Special (Mexico)',
       'Immigrants (Latin America)', 'Party Member (Colombia)',
       'Tijua

In [15]:
icews_cartels.to_csv('../data/icews_final.csv')

In [16]:
# what to do with events without a longitude latitude